In [36]:
import tensorflow as tf
import numpy as np
import matplotlib
import os
import matplotlib.pyplot as plt
from glob import glob
from os import path
from pandas import read_csv, concat
from csv import writer

In [49]:
tf.set_random_seed(777)

In [50]:
def MinMaxScaler(data):
    #데이터 모든 숫자들을 최솟 값만큼 뺀다.
    numerator = data - np.min(data,0)
    #최대값과 최소 값의 차이(A)를 구한다
    denominator = np.max(data,0)-np.min(data,0)
    #너무 큰 값이 나오지 않도록 나눈다
    return numerator / denominator(1e-7)

In [51]:
#하이퍼파라미터
seq_length = 140 #1개 시퀀스의 길이(시계열데이터 입력 개수)
data_dim = 3 #var개수
hidden_dim = 20 #각 셀의 출력 크기
output_dim = 2 #결과 분류 총 수
learning_rate = 0.01 #학습률
epoch_num = 20 #에폭 횟수 (학습용전체 데이터 반복횟수)

In [52]:
#데이터 로딩
#load data
folder_path = './csv/Angular_swing30/'

#파일 형식: s1_gyr.csv
swing_files = glob(path.join(folder_path, 's*_gyr.csv'))

df_swing = concat((read_csv(f,header=0) for f in swing_files),ignore_index=True)
df_swing.fillna(0, inplace=True)

#load data
folder_path = './csv/Angular_perting30/'

#파일 형식: s1_gyr.csv
perting_files2 = glob(path.join(folder_path, 'p*_gyr.csv'))

df_perting = concat((read_csv(f,header=0) for f in perting_files2),ignore_index=True)
df_perting.fillna(0, inplace=True)

df_swing.fillna(0, inplace=True)
df_perting.fillna(0, inplace=True)

In [53]:
def resizedataseq(df,seqlen):
    totlen = len(df)
    Qx = list(df.iloc[-1:-6:-1,0])
    Qy = list(df.iloc[-1:-6:-1,1])
    Qz = list(df.iloc[-1:-6:-1,2])
    
    while totlen != seqlen:
        mx = np.mean(Qx)
        my = np.mean(Qy)
        mz = np.mean(Qz)

        if totlen <seqlen:
            Qx.pop(0)
            Qy.pop(0)
            Qz.pop(0)
            Qx.append(mx)
            Qy.append(my)
            Qz.append(mz)
            #행추가
            df.loc[totlen]=[mx,my,mz]
            totlen = totlen+1
        
        
        elif totlen > seqlen:
            #행삭제
            df = df.iloc[:seqlen,:]
            totlen = seqlen
            break
            
    return df

In [54]:
X = np.empty(shape=(1,140,3))
y = []
#swing은 1 perting은 0
start = 1
end = 0

for i in range(1,len(df_swing)):
    if i==len(df_swing)-1:
        end = i-1
        subset= df_swing.iloc[start:end,1:].astype(float)
        subset= resizedataseq(subset,140)
        start = i+1
        if len(subset) == 140:
            subset = subset.values
            subset=subset[np.newaxis]
            X =np.concatenate((X,subset))
            y.append(1)

            
    if df_swing['time'][i] == 'Started:':
        end = i-1
        subset= df_swing.iloc[start:end,1:].astype(float)
        subset= resizedataseq(subset,140)
        start = i+1
        if len(subset) == 140:
            subset = subset.values
            subset=subset[np.newaxis]
            X =np.concatenate((X,subset))
            y.append(1)
        
print(X.shape)

(30, 140, 3)


In [55]:
X=np.delete(X,0,axis=0)

In [56]:
#swing은 1 perting은 0
start = 1
end = 0

for i in range(1,len(df_perting)):
    if i==len(df_perting)-1:
        end = i-1
        subset= df_perting.iloc[start:end,1:].astype(float)
        subset= resizedataseq(subset,140)
        start = i+1
        if len(subset) == 140:
            subset = subset.values
            subset=subset[np.newaxis]
            X =np.concatenate((X,subset))
            y.append(0)

            
    if df_perting['time'][i] == 'Started:':
        end = i-1
        subset= df_perting.iloc[start:end,1:].astype(float)
        subset= resizedataseq(subset,140)
        start = i+1
        if len(subset) == 140:
            subset = subset.values
            subset=subset[np.newaxis]
            X =np.concatenate((X,subset))
            y.append(0)
        
        
print(X.shape)

(57, 140, 3)


### X,y 묶어서  shffle 과정추가

In [57]:
#학습용.테스트용 데이터 생성
#80%를 학습용 데이터로 사용
train_size = int(len(X)*0.8)
#나머지 30%를 테스트용 데이터로 사용
test_size = len(X) - train_size

#데이터를 잘라 학습용 데이터 생성
trainX = np.array(X[0:train_size])
trainY = np.array(y[0:train_size])

#데이터를 잘라 테스트용 데이터 생성
testX  = np.array(X[train_size:len(X)])
testY = np.array(y[train_size:len(X)])

In [46]:
#텐서플로우 플레이스홀더 생성
#학습용/테스트용으로 X,Y를 생성한다
X = tf.placeholder(tf.float32,[None,seq_length,data_dim])
print("X:",X)
Y = tf.placeholder(tf.float32,[None,2])
print("Y:",Y)

#검증용 측정지표를 산출하기 위한 targets, predictions를 생성한다
targets = tf.placeholder(tf.float32,[None,1])
print("tragets",targets)
predictions = tf.placeholder(tf.float32,[None,1])
print("predictions:",predictions)

X: Tensor("Placeholder_8:0", shape=(?, 140, 3), dtype=float32)
Y: Tensor("Placeholder_9:0", shape=(?, 2), dtype=float32)
tragets Tensor("Placeholder_10:0", shape=(?, 1), dtype=float32)
predictions: Tensor("Placeholder_11:0", shape=(?, 1), dtype=float32)


In [47]:
#모델 (LSTM 네트워크) 생성
def lstm_cell():
    #LSTM셀을 생성한다
    #num_units:각 Cell 출력 크기
    #forget_bias: The bias added to forget gates.
    #state_is_tuple:True ==> accepted and returned states are 2-tuples of the c_state and m_state
    #state_is_tuple:False=>they are concatenated along the column axis.
    #cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_dim,state_is_tuple=True, activation = tf.sigmoid)
    #cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim,state_is_tuple_True, activation=tf.tanh)
    #relu,sigmoid
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim,forget_bias=0.8,
                                   state_is_tuple=True,activation=tf.sigmoid)
    return cell


In [62]:

num_stacked_layers = 1
stackedRNNs = [lstm_cell() for _ in range(1)]
multi_cells = tf.conrib.rnn.MultiRNNCell(stackedRNNs, state_is_tuple=True) if num_stacked_layers > 1 else lstm_cell()

# RNN Cell들을 연결
hypothesis, _states = tf.nn.dynamic_rnn(multi_cells, X, dtype= tf.float32)
print("hypothesis:",hypothesis)

hypothesis = tf.contrib.layers,fully_connected(hypothesis[:,-1]
                                              ,output_data_column_cnt
                                              ,activation_fn=tf.identity)

TypeError: in converted code:
    relative to C:\Users\Bin\Anaconda3\envs\venv_ml\lib\site-packages\tensorflow\python:

    ops\rnn_cell_impl.py:766 call *
        gate_inputs = math_ops.matmul(
    util\dispatch.py:180 wrapper
        return target(*args, **kwargs)
    ops\array_ops.py:1299 concat
        return gen_array_ops.concat_v2(values=values, axis=axis, name=name)
    ops\gen_array_ops.py:1255 concat_v2
        "ConcatV2", values=values, axis=axis, name=name)
    framework\op_def_library.py:499 _apply_op_helper
        raise TypeError("%s that don't all match." % prefix)

    TypeError: Tensors in list passed to 'values' of 'ConcatV2' Op have types [float64, float32] that don't all match.


In [48]:
# 몇개의 층으로 쌓인 Stacked RNNs 생성, 여기서는 1개층만
multi_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(1)]
                                          , state_is_tuple=True)

# RNN Cell(여기서는 LSTM셀임)들을 연결
#tf.nn.dynamic_rnn(mu)
hypothesis, _states = tf.nn.dynamic_rnn(multi_cells, X, dtype=tf.float32)
print("hypothesis: ", hypothesis)

Y_pred = tf.contrib.layers.fully_connected(hypothesis[:, -1]
                                           , output_dim, activation_fn=tf.sigmoid)

ValueError: in converted code:
    relative to C:\Users\Bin\Anaconda3\envs\venv_ml\lib\site-packages\tensorflow\python:

    ops\rnn_cell_impl.py:1719 call *
        cur_inp, new_state = cell(cur_inp, cur_state)
    ops\rnn_cell_impl.py:385 __call__
        self, inputs, state, scope=scope, *args, **kwargs)
    layers\base.py:537 __call__
        outputs = super(Layer, self).__call__(inputs, *args, **kwargs)
    keras\engine\base_layer.py:591 __call__
        self._maybe_build(inputs)
    keras\engine\base_layer.py:1881 _maybe_build
        self.build(input_shapes)
    keras\utils\tf_utils.py:295 wrapper
        output_shape = fn(instance, input_shape)
    ops\rnn_cell_impl.py:734 build
        shape=[input_depth + h_depth, 4 * self._num_units])
    keras\engine\base_layer.py:1484 add_variable
        return self.add_weight(*args, **kwargs)
    layers\base.py:450 add_weight
        **kwargs)
    keras\engine\base_layer.py:384 add_weight
        aggregation=aggregation)
    training\tra

## hidden_dimension 수정

In [34]:
loss = tf.reduce_sum(tf.square(Y_pred-Y))
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

#RMSE (Root Mean Sqare Error)
#rmse = tf.sqrt(tf.reduce_mean(tf.sqare(targetsw-predictions))) 
rmse = tf.sqrt(tf.reduce_mean(tf.squared_difference(targets,predictions))) 

In [35]:
with tf.Session() as sess:
    init= tf.global_variables_initializer()
    sess.run(init)
    
    #학습
    for epoch in range(epoch_num):
        _, step_loss = sess.run([train,loss],feed_dict={X:trainX, Y:trainY})
        print("[step: {} loss:: {}]".format(epoch, step_loss))
        
    #테스트한다
    test_predict = sess.run(Y_pred, feed_dict = {X:textX})
    
    #테스트용 데이터 기준으로 측정지표 remse를 산출한다
    rmse_val = sess.run(rmse, feed_dict = {targets:testY, predictions: test_predict})
    print("rmse:",rmse_val)
    
    plt.plot(textY,'r')
    plt.plot(test_predict, 'b')
    plt.xlabel("Time Period")
    plt.ylabel("Stock Price")
    plt.show()

ValueError: Cannot feed value of shape (45,) for Tensor 'Placeholder_5:0', which has shape '(?, 2)'